In [3]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Carregar o arquivo CSV
df = pd.read_csv(r"C:\Users\levic\Desktop\cienciaDeDados\exercicio02 - esse pra entrega\vendas_modificado.csv")

# Ordenar pelo id_da_compra
df = df.sort_values(by="id_da_compra")

# Selecionar apenas as primeiras 1000 linhas
df = df.head(1000)

# Exibir as primeiras 20 linhas
display(df.head(20))

# Exibir informações sobre os dados
df.info()


id_da_compra        data      hora                      cliente  \
119876             0  2019-07-07  11:05:44            PEDRO SILVA RAMOS   
143431             0  2019-07-07  11:05:44            PEDRO SILVA RAMOS   
4853               0  2019-07-07  11:05:44            PEDRO SILVA RAMOS   
364791             0  2019-07-07  11:05:44            PEDRO SILVA RAMOS   
170364             1  2018-08-23  17:52:53   PEDRO DA SILVA BIANQUINI     
102491             1  2018-08-23  17:52:53     PEDRO DA SILVA BIANQUINI   
219754             1  2018-08-23  17:52:53     PEDRO DA SILVA BIANQUINI   
158811             1  2018-08-23  17:52:53     PEDRO DA SILVA BIANQUINI   
40805              2  2022-11-06  07:03:58      LUCAS CUNHA DE OLIVEIRA   
86920              2  2022-11-06  07:03:58      LUCAS CUNHA DE OLIVEIRA   
218038             2  2022-11-06  07:03:58    LUCAS CUNHA DE OLIVEIRA     
1154               2  2022-11-06  07:03:58      LUCAS CUNHA DE OLIVEIRA   
52779              2  2022-11-06  07:03:58      LUCAS CUNHA DE OLIVEIRA   
125689             3  2021-03-05  22:45:31        PEDRO CARLOS DE SOUZA   
352557             3  2021-03-05  22:45:31        PEDRO CARLOS DE SOUZA   
320566             3  2021-03-05  22:45:31        PEDRO CARLOS DE SOUZA   
355670             3  2021-03-05  22:45:31        PEDRO CARLOS DE SOUZA   
99031              3  2021-03-05  22:45:31        PEDRO CARLOS DE SOUZA   
114733             3  2021-03-05  22:45:31        PEDRO CARLOS DE SOUZA   
5944               3  2021-03-05  22:45:31        PEDRO CARLOS DE SOUZA   

                 produto                  valor  quantidade   total  \
119876          Sabonete  R$ 288,20000000000005           7   25.40   
143431          Sabonete                 R$ 2,4          15   46.00   
4853            Macarrão                R$ 2,63          14   46.82   
364791    Pasta de Dente                R$ 2,98           9   36.82   
170364          Sabonete                R$ 2,37           9   21.33   
102491        Detergente                R$ 3,45          15   51.75   
219754          Macarrão                R$ 3,23           8   25.84   
158811      Refrigerante                R$ 6,83           5   34.15   
40805       Água Mineral                 R$ 1,4          10   23.00   
86920           Macarrão                R$ 3,76           6   31.56   
218038            Açúcar                R$ 2,13           2   13.26   
1154             Shampoo               R$ 14,52          12  183.24   
52779               Café                R$ 11,1           4   53.40   
125689  Queijo Mussarela               R$ 16,09          13  209.17   
352557   Suco de Laranja                 R$ 5,5           7   38.50   
320566      Óleo de Soja                R$ 3,41           2    6.82   
355670             Vinho               R$ 35,87           6  215.22   
99031   Queijo Mussarela               R$ 18,05          15  270.75   
114733          Manteiga                R$ 8,58          10   85.80   
5944        Pão de Forma                R$ 5,23           3   15.69   

                      status           cidade estado    pais        cep  \
119876       Aguardando Pgto      Ouro Branco     MG  Brasil  36420-000   
143431  Aguardando Pagamento      Ouro Branco     MG  Brasil  36420-000   
4853    Aguardando Pagamento      Ouro Branco     MG  Brasil   3642XXXX   
364791  Aguardando Pagamento      Ouro Branco     MG  Brasil  36420-000   
170364  Pagamento Confirmado  Duque de Caxias     RJ  Brasil  25000-000   
102491  Pagamento Confirmado  Duque de Caxias     RJ  Brasil  25000-000   
219754  Pagamento Confirmado  Duque de Caxias     RJ  Brasil  25000-000   
158811  Pagamento Confirmado  Duque de Caxias     RJ  Brasil  25000-000   
40805   Aguardando Pagamento           Viçosa     MG  Brasil  36570-000   
86920   Aguardando Pagamento           Viçosa     MG  Brasil  36570-000   
218038  Aguardando Pagamento           Viçosa     MG  Brasil  36570-000   
1154    Aguardando Pagamento           V

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 119876 to 345830
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id_da_compra  1000 non-null   int64  
 1   data          1000 non-null   object 
 2   hora          1000 non-null   object 
 3   cliente       1000 non-null   object 
 4   produto       1000 non-null   object 
 5   valor         1000 non-null   object 
 6   quantidade    1000 non-null   int64  
 7   total         991 non-null    float64
 8   status        1000 non-null   object 
 9   cidade        1000 non-null   object 
 10  estado        1000 non-null   object 
 11  pais          1000 non-null   object 
 12  cep           1000 non-null   object 
 13  frete         984 non-null    float64
 14  pagamento     1000 non-null   object 
 15  vendedor      993 non-null    object 
 16  marca         1000 non-null   object 
dtypes: float64(2), int64(2), object(13)
memory usage: 140.6+ KB


In [5]:
#Existem colunas nas compras que não fazem sentido os valores variar se tratando da mesma compra em conjunto em duas linhas
# Criar um DataFrame para armazenar as compras com valores diferentes nas colunas
compras_com_diferencas = pd.DataFrame()

# Colunas que precisam ter valores consistentes dentro de cada id_da_compra
colunas_fixas = ["data", "hora", "cliente", "status", "cidade", "estado", "pais", "cep", "frete", "vendedor"]

# Para cada coluna fixa, corrigir os valores dentro de cada id_da_compra
for coluna in colunas_fixas:
    # Identificar o valor mais comum (moda) para cada id_da_compra
    correcoes = df.groupby("id_da_compra")[coluna].agg(lambda x: x.mode()[0] if not x.mode().empty else x.iloc[0])

    # Substituir valores errados pelo valor mais frequente da compra
    for id_compra, valor_correto in correcoes.items():
        df.loc[df["id_da_compra"] == id_compra, coluna] = valor_correto

# Exibir os dados corrigidos
display(df)




id_da_compra        data      hora                     cliente  \
119876             0  2019-07-07  11:05:44           PEDRO SILVA RAMOS   
143431             0  2019-07-07  11:05:44           PEDRO SILVA RAMOS   
4853               0  2019-07-07  11:05:44           PEDRO SILVA RAMOS   
364791             0  2019-07-07  11:05:44           PEDRO SILVA RAMOS   
170364             1  2018-08-23  17:52:53    PEDRO DA SILVA BIANQUINI   
...              ...         ...       ...                         ...   
249416           166  2019-02-08  17:16:04  MATEUS VITOR FAÇANHA NEVES   
235575           166  2019-02-08  17:16:04  MATEUS VITOR FAÇANHA NEVES   
350503           166  2019-02-08  17:16:04  MATEUS VITOR FAÇANHA NEVES   
13164            166  2019-02-08  17:16:04  MATEUS VITOR FAÇANHA NEVES   
345830           166  2019-02-08  17:16:04  MATEUS VITOR FAÇANHA NEVES   

               produto                  valor  quantidade  total  \
119876        Sabonete  R$ 288,20000000000005           7  25.40   
143431        Sabonete                 R$ 2,4          15  46.00   
4853          Macarrão                R$ 2,63          14  46.82   
364791  Pasta de Dente                R$ 2,98           9  36.82   
170364        Sabonete                R$ 2,37           9  21.33   
...                ...                    ...         ...    ...   
249416        Presunto               R$ 10,05           5  60.75   
235575          Feijão                R$ 8,18           1  18.68   
350503          Açúcar                R$ 3,46           1  13.96   
13164          Shampoo                R$ 6,27          11  79.47   
345830  Pasta de Dente                R$ 3,37           4  23.98   

                      status           cidade estado    pais        cep  \
119876  Aguardando Pagamento      Ouro Branco     MG  Brasil  36420-000   
143431  Aguardando Pagamento      Ouro Branco     MG  Brasil  36420-000   
4853    Aguardando Pagamento      Ouro Branco     MG  Brasil  36420-000   
364791  Aguardando Pagamento      Ouro Branco     MG  Brasil  36420-000   
170364  Pagamento Confirmado  Duque de Caxias     RJ  Brasil  25000-000   
...                      ...              ...    ...     ...        ...   
249416         Em Transporte        Congonhas     MG  Brasil  36415-000   
235575         Em Transporte        Congonhas     MG  Brasil  36415-000   
350503         Em Transporte        Congonhas     MG  Brasil  36415-000   
13164          Em Transporte        Congonhas     MG  Brasil  36415-000   
345830         Em Transporte        Congonhas     MG  Brasil  36415-000   

        frete          pagamento                     vendedor      marca  
119876   10.0  Cartão de Crédito    VICTOR GONÇALVES DONADONI        Lux  
143431   10.0  Cartão de Crédito    VICTOR GONÇALVES DONADONI      Nivea  
4853     10.0  Cartão de Crédito    VICTOR GONÇALVES DONADONI     Renata  
364791   10.0  Cartão de Crédito    VICTOR GONÇALVES DONADONI    Colgate  
170364    0.0  Cartão de Crédito         HENRICO VICTOR ALVES       Dove  
...       ...                ...                          ...        ...  
249416   10.5                Pix  HENRICO MATOS LIMA DA CUNHA   Perdigão  
235575   10.5                Pix  HENRICO MATOS LIMA DA CUNHA  Caldo Bom  
350503   10.5                Pix  HENRICO MATOS LIMA DA CUNHA  Caravelas  
13164    10.5                Pix  HENRICO MATOS LIMA DA CUNHA    Pantene  
345830   10.5                Pix  HENRICO MATOS LIMA DA CUNHA    Colgate  

[1000 rows x 17 columns]

In [6]:

# Remover caracteres especiais, mantendo apenas letras e espaços
df["produto"] = df["produto"].str.replace(r'[^a-zA-ZáéíóúãõçÁÉÍÓÚÃÕÇ\s]', '', regex=True)

display(df.head(20))


id_da_compra        data      hora                   cliente  \
119876             0  2019-07-07  11:05:44         PEDRO SILVA RAMOS   
143431             0  2019-07-07  11:05:44         PEDRO SILVA RAMOS   
4853               0  2019-07-07  11:05:44         PEDRO SILVA RAMOS   
364791             0  2019-07-07  11:05:44         PEDRO SILVA RAMOS   
170364             1  2018-08-23  17:52:53  PEDRO DA SILVA BIANQUINI   
102491             1  2018-08-23  17:52:53  PEDRO DA SILVA BIANQUINI   
219754             1  2018-08-23  17:52:53  PEDRO DA SILVA BIANQUINI   
158811             1  2018-08-23  17:52:53  PEDRO DA SILVA BIANQUINI   
40805              2  2022-11-06  07:03:58   LUCAS CUNHA DE OLIVEIRA   
86920              2  2022-11-06  07:03:58   LUCAS CUNHA DE OLIVEIRA   
218038             2  2022-11-06  07:03:58   LUCAS CUNHA DE OLIVEIRA   
1154               2  2022-11-06  07:03:58   LUCAS CUNHA DE OLIVEIRA   
52779              2  2022-11-06  07:03:58   LUCAS CUNHA DE OLIVEIRA   
125689             3  2021-03-05  22:45:31     PEDRO CARLOS DE SOUZA   
352557             3  2021-03-05  22:45:31     PEDRO CARLOS DE SOUZA   
320566             3  2021-03-05  22:45:31     PEDRO CARLOS DE SOUZA   
355670             3  2021-03-05  22:45:31     PEDRO CARLOS DE SOUZA   
99031              3  2021-03-05  22:45:31     PEDRO CARLOS DE SOUZA   
114733             3  2021-03-05  22:45:31     PEDRO CARLOS DE SOUZA   
5944               3  2021-03-05  22:45:31     PEDRO CARLOS DE SOUZA   

                 produto                  valor  quantidade   total  \
119876          Sabonete  R$ 288,20000000000005           7   25.40   
143431          Sabonete                 R$ 2,4          15   46.00   
4853            Macarrão                R$ 2,63          14   46.82   
364791    Pasta de Dente                R$ 2,98           9   36.82   
170364          Sabonete                R$ 2,37           9   21.33   
102491        Detergente                R$ 3,45          15   51.75   
219754          Macarrão                R$ 3,23           8   25.84   
158811      Refrigerante                R$ 6,83           5   34.15   
40805       Água Mineral                 R$ 1,4          10   23.00   
86920           Macarrão                R$ 3,76           6   31.56   
218038            Açúcar                R$ 2,13           2   13.26   
1154             Shampoo               R$ 14,52          12  183.24   
52779               Café                R$ 11,1           4   53.40   
125689  Queijo Mussarela               R$ 16,09          13  209.17   
352557   Suco de Laranja                 R$ 5,5           7   38.50   
320566      Óleo de Soja                R$ 3,41           2    6.82   
355670             Vinho               R$ 35,87           6  215.22   
99031   Queijo Mussarela               R$ 18,05          15  270.75   
114733          Manteiga                R$ 8,58          10   85.80   
5944        Pão de Forma                R$ 5,23           3   15.69   

                      status           cidade estado    pais        cep  \
119876  Aguardando Pagamento      Ouro Branco     MG  Brasil  36420-000   
143431  Aguardando Pagamento      Ouro Branco     MG  Brasil  36420-000   
4853    Aguardando Pagamento      Ouro Branco     MG  Brasil  36420-000   
364791  Aguardando Pagamento      Ouro Branco     MG  Brasil  36420-000   
170364  Pagamento Confirmado  Duque de Caxias     RJ  Brasil  25000-000   
102491  Pagamento Confirmado  Duque de Caxias     RJ  Brasil  25000-000   
219754  Pagamento Confirmado  Duque de Caxias     RJ  Brasil  25000-000   
158811  Pagamento Confirmado  Duque de Caxias     RJ  Brasil  25000-000   
40805   Aguardando Pagamento           Viçosa     MG  Brasil  36570-000   
86920   Aguardando Pagamento           Viçosa     MG  Brasil  36570-000   
218038  Aguardando Pagamento           Viçosa     MG  Brasil  36570-000   
1154    Aguardando Pagamento           Viçosa     MG  Brasil  36570-000   
52779   Aguardando Pagamento

In [7]:
# A coluna "valor" dos produtos possuem casas decimais inconsistentes, além de estarem no formato de string
#vou remover o R$, definir para float, e colocar como 2 casas decimais após o "." para ficar padronizado.

# Remover espaços extras antes e depois do valor
df['valor'] = df['valor'].str.strip()

# Remover "R$" e possíveis espaços
df['valor'] = df['valor'].str.replace(r'R\$', '', regex=True).str.strip()

# Substituir "," por "." e converter para float
df['valor'] = df['valor'].str.replace(',', '.', regex=True).astype(float)

# Arredondar para duas casas decimais
df['valor'] = df['valor'].round(2)

# Exibir para conferir
display(df.head(15))


id_da_compra        data      hora                   cliente  \
119876             0  2019-07-07  11:05:44         PEDRO SILVA RAMOS   
143431             0  2019-07-07  11:05:44         PEDRO SILVA RAMOS   
4853               0  2019-07-07  11:05:44         PEDRO SILVA RAMOS   
364791             0  2019-07-07  11:05:44         PEDRO SILVA RAMOS   
170364             1  2018-08-23  17:52:53  PEDRO DA SILVA BIANQUINI   
102491             1  2018-08-23  17:52:53  PEDRO DA SILVA BIANQUINI   
219754             1  2018-08-23  17:52:53  PEDRO DA SILVA BIANQUINI   
158811             1  2018-08-23  17:52:53  PEDRO DA SILVA BIANQUINI   
40805              2  2022-11-06  07:03:58   LUCAS CUNHA DE OLIVEIRA   
86920              2  2022-11-06  07:03:58   LUCAS CUNHA DE OLIVEIRA   
218038             2  2022-11-06  07:03:58   LUCAS CUNHA DE OLIVEIRA   
1154               2  2022-11-06  07:03:58   LUCAS CUNHA DE OLIVEIRA   
52779              2  2022-11-06  07:03:58   LUCAS CUNHA DE OLIVEIRA   
125689             3  2021-03-05  22:45:31     PEDRO CARLOS DE SOUZA   
352557             3  2021-03-05  22:45:31     PEDRO CARLOS DE SOUZA   

                 produto   valor  quantidade   total                status  \
119876          Sabonete  288.20           7   25.40  Aguardando Pagamento   
143431          Sabonete    2.40          15   46.00  Aguardando Pagamento   
4853            Macarrão    2.63          14   46.82  Aguardando Pagamento   
364791    Pasta de Dente    2.98           9   36.82  Aguardando Pagamento   
170364          Sabonete    2.37           9   21.33  Pagamento Confirmado   
102491        Detergente    3.45          15   51.75  Pagamento Confirmado   
219754          Macarrão    3.23           8   25.84  Pagamento Confirmado   
158811      Refrigerante    6.83           5   34.15  Pagamento Confirmado   
40805       Água Mineral    1.40          10   23.00  Aguardando Pagamento   
86920           Macarrão    3.76           6   31.56  Aguardando Pagamento   
218038            Açúcar    2.13           2   13.26  Aguardando Pagamento   
1154             Shampoo   14.52          12  183.24  Aguardando Pagamento   
52779               Café   11.10           4   53.40  Aguardando Pagamento   
125689  Queijo Mussarela   16.09          13  209.17  Pagamento Confirmado   
352557   Suco de Laranja    5.50           7   38.50  Pagamento Confirmado   

                 cidade estado    pais        cep  frete  \
119876      Ouro Branco     MG  Brasil  36420-000   10.0   
143431      Ouro Branco     MG  Brasil  36420-000   10.0   
4853        Ouro Branco     MG  Brasil  36420-000   10.0   
364791      Ouro Branco     MG  Brasil  36420-000   10.0   
170364  Duque de Caxias     RJ  Brasil  25000-000    0.0   
102491  Duque de Caxias     RJ  Brasil  25000-000    0.0   
219754  Duque de Caxias     RJ  Brasil  25000-000    0.0   
158811  Duque de Caxias     RJ  Brasil  25000-000    0.0   
40805            Viçosa     MG  Brasil  36570-000    9.0   
86920            Viçosa     MG  Brasil  36570-000    9.0   
218038           Viçosa     MG  Brasil  36570-000    9.0   
1154             Viçosa     MG  Brasil  36570-000    9.0   
52779            Viçosa     MG  Brasil  36570-000    9.0   
125689      São Gonçalo     RJ  Brasil  24400-000    0.0   
352557      São Gonçalo     RJ  Brasil  24400-000    0.0   

                     pagamento                              vendedor    marca  
119876       Cartão de Crédito             VICTOR GONÇALVES DONADONI      Lux  
143431       Cartão de Crédito             VICTOR GONÇALVES DONADONI    Nivea  
4853         Cartão de Crédito             VICTOR GONÇALVES DONADONI   Renata  
364791       Cartão de Crédito             VICTOR GONÇALVES DONADONI  Colgate  
170364       Cartão de Crédito                  HENRICO VICTOR ALVES     Dove  
102491       Cartão de Crédito                  HENRICO VICTOR ALVES  Minuano  
219754       Cartão de Crédito                  HENRICO VICTOR ALVES  Barilla  
15881

In [8]:
# Verificar a existencia de valores nulos
display(df.isnull().sum())

# Identificamos a coluna "total" com valores nulos, para resolver já vamos confirmar se todos os outros estão certos
# multiplicando a coluna "valor" vezes "quantidade"
df['total'] = df['valor'] * df['quantidade'] + df['frete']

display(df.head(15)) 

id_da_compra    0
data            0
hora            0
cliente         0
produto         0
valor           0
quantidade      0
total           9
status          0
cidade          0
estado          0
pais            0
cep             0
frete           0
pagamento       0
vendedor        0
marca           0
dtype: int64

id_da_compra        data      hora                   cliente  \
119876             0  2019-07-07  11:05:44         PEDRO SILVA RAMOS   
143431             0  2019-07-07  11:05:44         PEDRO SILVA RAMOS   
4853               0  2019-07-07  11:05:44         PEDRO SILVA RAMOS   
364791             0  2019-07-07  11:05:44         PEDRO SILVA RAMOS   
170364             1  2018-08-23  17:52:53  PEDRO DA SILVA BIANQUINI   
102491             1  2018-08-23  17:52:53  PEDRO DA SILVA BIANQUINI   
219754             1  2018-08-23  17:52:53  PEDRO DA SILVA BIANQUINI   
158811             1  2018-08-23  17:52:53  PEDRO DA SILVA BIANQUINI   
40805              2  2022-11-06  07:03:58   LUCAS CUNHA DE OLIVEIRA   
86920              2  2022-11-06  07:03:58   LUCAS CUNHA DE OLIVEIRA   
218038             2  2022-11-06  07:03:58   LUCAS CUNHA DE OLIVEIRA   
1154               2  2022-11-06  07:03:58   LUCAS CUNHA DE OLIVEIRA   
52779              2  2022-11-06  07:03:58   LUCAS CUNHA DE OLIVEIRA   
125689             3  2021-03-05  22:45:31     PEDRO CARLOS DE SOUZA   
352557             3  2021-03-05  22:45:31     PEDRO CARLOS DE SOUZA   

                 produto   valor  quantidade    total                status  \
119876          Sabonete  288.20           7  2027.40  Aguardando Pagamento   
143431          Sabonete    2.40          15    46.00  Aguardando Pagamento   
4853            Macarrão    2.63          14    46.82  Aguardando Pagamento   
364791    Pasta de Dente    2.98           9    36.82  Aguardando Pagamento   
170364          Sabonete    2.37           9    21.33  Pagamento Confirmado   
102491        Detergente    3.45          15    51.75  Pagamento Confirmado   
219754          Macarrão    3.23           8    25.84  Pagamento Confirmado   
158811      Refrigerante    6.83           5    34.15  Pagamento Confirmado   
40805       Água Mineral    1.40          10    23.00  Aguardando Pagamento   
86920           Macarrão    3.76           6    31.56  Aguardando Pagamento   
218038            Açúcar    2.13           2    13.26  Aguardando Pagamento   
1154             Shampoo   14.52          12   183.24  Aguardando Pagamento   
52779               Café   11.10           4    53.40  Aguardando Pagamento   
125689  Queijo Mussarela   16.09          13   209.17  Pagamento Confirmado   
352557   Suco de Laranja    5.50           7    38.50  Pagamento Confirmado   

                 cidade estado    pais        cep  frete  \
119876      Ouro Branco     MG  Brasil  36420-000   10.0   
143431      Ouro Branco     MG  Brasil  36420-000   10.0   
4853        Ouro Branco     MG  Brasil  36420-000   10.0   
364791      Ouro Branco     MG  Brasil  36420-000   10.0   
170364  Duque de Caxias     RJ  Brasil  25000-000    0.0   
102491  Duque de Caxias     RJ  Brasil  25000-000    0.0   
219754  Duque de Caxias     RJ  Brasil  25000-000    0.0   
158811  Duque de Caxias     RJ  Brasil  25000-000    0.0   
40805            Viçosa     MG  Brasil  36570-000    9.0   
86920            Viçosa     MG  Brasil  36570-000    9.0   
218038           Viçosa     MG  Brasil  36570-000    9.0   
1154             Viçosa     MG  Brasil  36570-000    9.0   
52779            Viçosa     MG  Brasil  36570-000    9.0   
125689      São Gonçalo     RJ  Brasil  24400-000    0.0   
352557      São Gonçalo     RJ  Brasil  24400-000    0.0   

                     pagamento                              vendedor    marca  
119876       Cartão de Crédito             VICTOR GONÇALVES DONADONI      Lux  
143431       Cartão de Crédito             VICTOR GONÇALVES DONADONI    Nivea  
4853         Cartão de Crédito             VICTOR GONÇALVES DONADONI   Renata  
364791       Cartão de Crédito             VICTOR GONÇALVES DONADONI  Colgate  
170364       Cartão de Crédito                  HENRICO VICTOR ALVES     Dove  
102491       Cartão de Crédito                  HENRICO VICTOR ALVES  Minuano  
219754       Cartão de Crédito                  HENRICO VICTOR ALVES 

In [9]:
# Confirmando se todas as datas e horas estão no mesmo formato
df['data'] = pd.to_datetime(df['data'], errors='coerce')

# Padronizar hora
df['hora'] = pd.to_datetime(df['hora'], format='%H:%M:%S', errors='coerce').dt.time



In [10]:
# Seleciona apenas colunas numéricas
colunas_numericas = df.select_dtypes(include='number').columns

# Listas para armazenar os resultados
colunas_com_negativos = []
colunas_sem_negativos = []

# Verifica cada coluna
for col in colunas_numericas:
    if (df[col] < 0).any():
        colunas_com_negativos.append(col)
        print(f"Coluna '{col}' tem valores negativos:")
        print(df[df[col] < 0][[col]].head())
        print()
    else:
        colunas_sem_negativos.append(col)


df[colunas_numericas] = df[colunas_numericas].abs()
#agora todos os valores que estavam negativos foram transformados em positivos

In [11]:
# vamos identificar e tratar os dados em outlier
def tratar_outliers_por_produto(df, col_produto='produto', col_valor='valor'):
    df = df.copy()
    resumo_outliers = []  # Armazena tuplas: (produto, total, outliers, %)

    for produto, grupo in df.groupby(col_produto):
        q1 = grupo[col_valor].quantile(0.25)
        q3 = grupo[col_valor].quantile(0.75)
        iqr = q3 - q1
        limite_inferior = q1 - 1.5 * iqr
        limite_superior = q3 + 1.5 * iqr

        condicao_outlier = (df[col_produto] == produto) & (
            (df[col_valor] < limite_inferior) | (df[col_valor] > limite_superior)
        )

        outliers = df[condicao_outlier]
        total_produto = len(grupo)
        total_outliers = len(outliers)
        perc = (total_outliers / total_produto) * 100 if total_produto else 0

        if total_outliers > 0:
            print(f"\nProduto: {produto}")
            print(f"Limites: {limite_inferior:.2f} a {limite_superior:.2f}")
            print(f"Outliers encontrados: {total_outliers} de {total_produto} ({perc:.2f}%)")
            print(outliers[[col_produto, col_valor]])

        resumo_outliers.append({
            'produto': produto,
            'total': total_produto,
            'outliers': total_outliers,
            'percentual': round(perc, 2)
        })

        # Substitui os outliers por NaN
        df.loc[condicao_outlier, col_valor] = None

    # Cria DataFrame de resumo
    resumo_df = pd.DataFrame(resumo_outliers).sort_values(by='percentual', ascending=False)
    
    print("\nResumo dos outliers por produto:")
    print(resumo_df)

    return df

def preencher_valores_nulos_com_media(df, col_produto='produto', col_valor='valor'):
    df = df.copy()
    df[col_valor] = df.groupby(col_produto)[col_valor].transform(
        lambda x: x.fillna(x.mean())
    )
    return df


# 1. Substitui outliers por NaN
df = tratar_outliers_por_produto(df)

# 2. Preenche os NaN com a média de valor por produto
df = preencher_valores_nulos_com_media(df)




Produto: Açúcar
Limites: 1.70 a 4.25
Outliers encontrados: 1 de 56 (1.79%)
       produto   valor
196440  Açúcar  413.96

Produto: Biscoito Recheado
Limites: 2.17 a 3.74
Outliers encontrados: 1 de 29 (3.45%)
                  produto  valor
253620  Biscoito Recheado   3.77

Produto: Café
Limites: 5.17 a 14.81
Outliers encontrados: 1 de 50 (2.00%)
       produto    valor
368388    Café  1361.09

Produto: Condicionador
Limites: 4.19 a 16.70
Outliers encontrados: 2 de 24 (8.33%)
              produto    valor
185863  Condicionador   830.54
206628  Condicionador  1160.66

Produto: Desinfetante
Limites: 1.96 a 7.68
Outliers encontrados: 2 de 55 (3.64%)
             produto   valor
83243   Desinfetante  882.94
306947  Desinfetante  596.05

Produto: Farinha de Trigo
Limites: 1.52 a 4.57
Outliers encontrados: 1 de 20 (5.00%)
                 produto   valor
144442  Farinha de Trigo  358.94

Produto: Feijão
Limites: 6.62 a 10.10
Outliers encontrados: 3 de 26 (11.54%)
       produto  valor
2081

In [12]:
# vamos descobrir associações entre produtos
# Agrupa os produtos por id da compra
transacoes = df.groupby('id_da_compra')['produto'].apply(list).tolist()

# Transforma em formato binário para o Apriori
te = TransactionEncoder()
te_array = te.fit(transacoes).transform(transacoes)
df_binario = pd.DataFrame(te_array, columns=te.columns_)

# Aplica Apriori
frequent_itemsets = apriori(df_binario, min_support=0.02, use_colnames=True)

# Gera regras de associação
regras = association_rules(frequent_itemsets, metric='lift', min_threshold=1)

# Mostra as regras
print(regras[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

            antecedents                                        consequents  \
0           (Amaciante)                                           (Açúcar)   
1              (Açúcar)                                        (Amaciante)   
2           (Amaciante)                                   (Pasta de Dente)   
3      (Pasta de Dente)                                        (Amaciante)   
4           (Amaciante)                                         (Sabonete)   
..                  ...                                                ...   
933  (Queijo Mussarela)            (Feijão, Molho de Tomate, Café, Açúcar)   
934              (Café)  (Queijo Mussarela, Feijão, Molho de Tomate, Aç...   
935   (Molho de Tomate)           (Queijo Mussarela, Feijão, Açúcar, Café)   
936            (Açúcar)  (Queijo Mussarela, Feijão, Molho de Tomate, Café)   
937            (Feijão)  (Queijo Mussarela, Molho de Tomate, Café, Açúcar)   

      support  confidence      lift  
0    0.023952    0.333333